In [187]:
key_const = [2, -6, -1, 4, -3, 2, -5, 0, -7, -2, 3, -4, 2, -6, -1]

In [6]:
import mido as md
import py_midicsv

raw_song = py_midicsv.midi_to_csv('chopin/chpn-p8.mid')
raw_song2 = py_midicsv.midi_to_csv('chopin/chpn-p21.mid')
raw_song3 = py_midicsv.midi_to_csv('chopin/Silent_Night_-_Pentatonic_ver..mid')

raw_song = [e.strip().split(",") for e in raw_song]
raw_song2 = [e.strip().split(",") for e in raw_song2]
raw_song3 = [e.strip().split(",") for e in raw_song3]
for k in raw_song:
    print(k)

['0', ' 0', ' Header', ' 1', ' 8', ' 480']
['1', ' 0', ' Start_track']
['1', ' 0', ' Title_t', ' "Prelude No. 8"']
['1', ' 0', ' Copyright_t', ' "Copyright \\251 1997 by Bernd Krueger http://www.piano-midi.de"']
['1', ' 0', ' Text_t', ' "Frederic Chopin"']
['1', ' 0', ' Text_t', ' "Molto agitato"']
['1', ' 0', ' Text_t', ' "Fertiggestellt am 6.5.97\\012"']
['1', ' 0', ' Text_t', ' "Korrekturen am 27.8.98\\012"']
['1', ' 0', ' Text_t', ' "Anpassungen an Digitalpiano am 18.12.98\\012"']
['1', ' 0', ' Text_t', ' "Update am 11.4.99\\012"']
['1', ' 0', ' Text_t', ' "Normierung: 23.12.2002\\012"']
['1', ' 0', ' Text_t', ' "Update am 10.6.2010\\012"']
['1', ' 0', ' Text_t', ' "Update am 20.2.2014\\012"']
['1', ' 0', ' Text_t', ' "Dauer: 1:31 Minuten\\012"']
['1', ' 0', ' SMPTE_offset', ' 96', ' 0', ' 3', ' 0', ' 0']
['1', ' 0', ' Time_signature', ' 4', ' 2', ' 24', ' 8']
['1', ' 0', ' Key_signature', ' 3', ' "major"']
['1', ' 0', ' Tempo', ' 666667']
['1', ' 80', ' Tempo', ' 662983']
['1', ' 

['2', ' 55380', ' Note_on_c', ' 0', ' 68', ' 0']
['2', ' 55380', ' Note_on_c', ' 0', ' 71', ' 27']
['2', ' 55440', ' Note_on_c', ' 0', ' 71', ' 0']
['2', ' 55440', ' Note_on_c', ' 0', ' 70', ' 31']
['2', ' 55500', ' Note_on_c', ' 0', ' 70', ' 0']
['2', ' 55500', ' Note_on_c', ' 0', ' 66', ' 25']
['2', ' 55560', ' Note_on_c', ' 0', ' 66', ' 0']
['2', ' 55620', ' Note_on_c', ' 0', ' 73', ' 31']
['2', ' 55680', ' Note_on_c', ' 0', ' 73', ' 0']
['2', ' 55740', ' Note_on_c', ' 0', ' 75', ' 22']
['2', ' 55800', ' Note_on_c', ' 0', ' 75', ' 0']
['2', ' 55800', ' Note_on_c', ' 0', ' 70', ' 28']
['2', ' 55860', ' Note_on_c', ' 0', ' 70', ' 0']
['2', ' 55860', ' Note_on_c', ' 0', ' 73', ' 22']
['2', ' 55920', ' Note_on_c', ' 0', ' 73', ' 0']
['2', ' 55920', ' Note_on_c', ' 0', ' 71', ' 29']
['2', ' 55980', ' Note_on_c', ' 0', ' 71', ' 0']
['2', ' 55980', ' Note_on_c', ' 0', ' 66', ' 23']
['2', ' 56040', ' Note_on_c', ' 0', ' 66', ' 0']
['2', ' 56100', ' Note_on_c', ' 0', ' 75', ' 29']
['2', ' 56

['4', ' 34568', ' Control_c', ' 0', ' 64', ' 0']
['4', ' 34612', ' Control_c', ' 0', ' 64', ' 127']
['4', ' 34640', ' Note_on_c', ' 0', ' 57', ' 0']
['4', ' 34640', ' Note_on_c', ' 0', ' 49', ' 55']
['4', ' 34720', ' Note_on_c', ' 0', ' 49', ' 0']
['4', ' 34720', ' Note_on_c', ' 0', ' 54', ' 59']
['4', ' 34800', ' Note_on_c', ' 0', ' 54', ' 0']
['4', ' 34800', ' Note_on_c', ' 0', ' 42', ' 57']
['4', ' 35040', ' Note_on_c', ' 0', ' 42', ' 0']
['4', ' 35040', ' Note_on_c', ' 0', ' 57', ' 66']
['4', ' 35048', ' Control_c', ' 0', ' 64', ' 0']
['4', ' 35083', ' Control_c', ' 0', ' 64', ' 127']
['4', ' 35120', ' Note_on_c', ' 0', ' 57', ' 0']
['4', ' 35120', ' Note_on_c', ' 0', ' 49', ' 55']
['4', ' 35200', ' Note_on_c', ' 0', ' 49', ' 0']
['4', ' 35200', ' Note_on_c', ' 0', ' 54', ' 61']
['4', ' 35280', ' Note_on_c', ' 0', ' 54', ' 0']
['4', ' 35280', ' Note_on_c', ' 0', ' 45', ' 56']
['4', ' 35520', ' Note_on_c', ' 0', ' 45', ' 0']
['4', ' 35520', ' Note_on_c', ' 0', ' 62', ' 70']
['4', ' 

# Load data (Note, Key, TimeSignature)

In [7]:
import os
import pandas as pd

songs = []
keys = []
time_sig =[]
for filename in os.listdir('chopin'):
    
    
    song = py_midicsv.midi_to_csv("chopin/" + filename)
    song = [e.strip().split(", ") for e in song]
    
    divider = int(song[0][5])

    df = pd.DataFrame(song)
    
    keys.append(  (  int(df[ df[2] == "Key_signature"][3].values[0]), df[ df[2] == "Key_signature"][4].values[0])  )
    time_sig.append( int(df[ df[2] == "Time_signature"][3].values[0]) )
    
    df = df[ (df[2]=='Note_on_c') ]
    
    df[4] = df[4].astype(int)
    df[1] = df[1].astype(int)
    df[1] = df[1]/divider
    df = df.sort_values(by = [4,1] , ascending=True)
    df = df[[1,4] ]
    df = df.reset_index(drop=True)
    songs.append(df)
    
songs[4].head(100)

,1,4
0,168.0,30
1,168.5,30
2,210.0,30
3,210.5,30
4,216.0,30
...,...,...
95,78.5,42
96,81.0,42
97,81.5,42
98,102.0,42


# Calculate Note Diff

In [8]:
diff = []
for i in range(len(songs)):

    rng = range(0, len(songs[i]), 2)
    rng2 = range(1, len(songs[i]), 2)
    
    start_notes = songs[i].iloc[rng]
    stop_notes = songs[i].iloc[rng2]
    
    start_notes = start_notes.reset_index(drop=True)
    stop_notes = stop_notes.reset_index(drop=True)
    
     
    diff.append(stop_notes[1].subtract(start_notes[1]).to_frame()  )
type(diff[0])

pandas.core.frame.DataFrame

# Restruct Dataframe 

In [9]:
notes = []
for i in range(len(songs)):
    
    rng = range(0, len(songs[i]), 2)
    start_notes = songs[i].iloc[rng]
    
    start_notes = start_notes.reset_index(drop=True)
    diff[i].rename(columns = {1:2}, inplace = True)
    
    result = pd.concat([start_notes, diff[i]], axis=1, join='inner')
    result.rename(columns = {1:'start', 4:'pitch', 
                              2:'duration'}, inplace = True)
    result = result.sort_values(by = ['start','duration'] , ascending=True)
    notes.append(result)
notes[47].tail(10)

,start,pitch,duration
3230,1165.5000,65,1.5000
825,1168.1250,49,0.3750
1802,1168.1875,56,0.3125
2730,1168.2500,61,0.2500
3231,1168.2500,65,0.2500
3658,1168.2500,68,0.2500
4242,1168.2500,73,0.2500
15,1168.5000,25,1.5000
208,1168.5000,37,1.5000
5975,1168.5000,101,1.5000


# Contruct Melody Matrix

In [11]:
import numpy as np
import matplotlib.pyplot as plt
songs_chart = []

for i in range(len(notes) ):
    index = 0
    song_chart = []
    for bar in range (1,int(notes[i].iloc[-1]['start'])+1):
        if bar > 32:
            break
        arr = np.zeros([128, 96])
        while notes[i].iloc[index]['start'] < bar:
            start = (notes[i].iloc[index]['start']-bar+1)*96
            stop = start + (notes[i].iloc[index]['duration'])*96
            #print(bar,round(start,5),round(stop,5)-1)
            #print((notes[i].iloc[index]['start']-bar+1)*96,(notes[i].iloc[index]['duration'])*96,bar)
            
            arr[int(notes[i].iloc[index]['pitch'])][int(start):int(stop)-1] = np.ones(arr[int(notes[i].iloc[index]['pitch'])][int(start):int(stop)-1].shape)
            
            #print(arr2.shape)
            index+=1
        
        song_chart.append(arr)
    songs_chart.append(song_chart)
    print(i,len(song_chart))
       
#         print('cut')

#     print('cut_song')


0 32
1 32
2 32
3 32
4 32
5 32
6 32
7 32
8 32
9 32
10 32
11 32
12 32
13 32
14 32
15 32
16 32
17 32
18 32
19 32
20 32
21 32
22 32
23 32
24 32
25 32
26 32
27 32
28 32
29 32
30 32
31 32
32 32
33 32
34 32
35 32
36 32
37 32
38 32
39 32
40 32
41 32
42 32
43 32
44 32
45 32
46 32
47 32
48 32


In [26]:
for k in songs_chart:
    print(len(k))

32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32


# Model

In [165]:
import tensorflow as tf
x_train = []
y_train = []

for k in songs_chart:
    for i in range(len(k)-8):
        x_train.append(np.asarray(k[i:i+7]) )
        y_train.append(k[i+8] )

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
print(y_train.shape,x_train.shape)

(1176, 128, 96) (1176, 7, 128, 96)


In [170]:
x_train = np.reshape(x_train,(1176,7*128*96))
y_train = np.reshape(y_train,(1176,1*128*96))

In [184]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(7*128*96,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(128*96, activation='relu')
])
model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=1e-4),
        loss='mean_squared_error',
        metrics=['acc']
    )

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=32,verbose=1) 

In [140]:

1176/1176 [==============================] - 4s 3ms/sample - loss: 0.0182 - acc: 0.model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_33 (Flatten)         (None, 86016)             0         
_________________________________________________________________
dense_96 (Dense)             (None, 128)               11010176  
_________________________________________________________________
dense_97 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_98 (Dense)             (None, 12288)             1585152   
Total params: 12,611,840
Trainable params: 12,611,840
Non-trainable params: 0
_________________________________________________________________


# Decoding

In [220]:
index = 0
for bar in range (1,int(notes[3].iloc[-1]['start']) + 2):
        A = np.zeros([128, 97])
        while notes[3].iloc[index]['start'] < bar:
            print(notes[3].iloc[index]['start'], bar)
            index+=1
        print('cut')

0.0020833333333333333 1
0.0020833333333333333 1
0.0020833333333333333 1
0.0020833333333333333 1
0.0020833333333333333 1
0.25 1
0.5 1
0.5 1
0.5 1
0.5 1
0.75 1
cut
1.0 2
1.0 2
1.0 2
1.0 2
1.0 2
1.25 2
1.5 2
1.5 2
1.5 2
1.5 2
1.5 2
1.5 2
1.75 2
cut
2.0 3
2.0 3
2.0 3
2.0 3
2.0 3
2.25 3
2.5 3
2.5 3
2.5 3
2.5 3
2.5 3
2.75 3
cut
3.0 4
3.0 4
3.0 4
3.0 4
3.0 4
3.25 4
3.5 4
3.5 4
3.5 4
3.5 4
3.75 4
cut
4.0 5
4.0 5
4.0 5
4.0 5
4.0 5
4.25 5
4.5 5
4.5 5
4.5 5
4.5 5
4.5 5
4.5 5
4.75 5
cut
5.0 6
5.0 6
5.0 6
5.0 6
5.0 6
5.25 6
5.5 6
5.5 6
5.5 6
5.5 6
5.5 6
5.75 6
cut
6.0 7
6.0 7
6.0 7
6.0 7
6.0 7
6.25 7
6.5 7
6.5 7
6.5 7
6.5 7
6.75 7
cut
7.0 8
7.0 8
7.0 8
7.0 8
7.0 8
7.25 8
7.5 8
7.5 8
7.5 8
7.5 8
7.5 8
7.75 8
cut
8.0 9
8.0 9
8.0 9
8.0 9
8.0 9
8.25 9
8.5 9
8.5 9
8.5 9
8.5 9
8.5 9
8.75 9
cut
9.0 10
9.0 10
9.0 10
9.0 10
9.0 10
9.25 10
9.5 10
9.5 10
9.5 10
9.5 10
9.5 10
9.75 10
cut
10.0 11
10.0 11
10.0 11
10.0 11
10.0 11
10.25 11
10.5 11
10.5 11
10.5 11
10.75 11
cut
11.0 12
11.0 12
11.0 12
11.0 12
11.25 

99.0 100
99.0 100
99.25 100
99.5 100
99.5 100
99.5 100
99.5 100
99.75 100
cut
100.0 101
100.0 101
100.0 101
100.0 101
100.0 101
100.25 101
100.5 101
100.5 101
100.5 101
100.5 101
100.5 101
100.75 101
cut
101.0 102
101.0 102
101.25 102
101.5 102
101.5 102
101.5 102
101.5 102
101.5 102
101.75 102
cut
102.0 103
102.0 103
102.0 103
102.0 103
102.5 103
102.5 103
102.5 103
102.5 103
102.5 103
102.75 103
cut
103.0 104
103.0 104
103.0 104
103.0 104
103.0 104
103.25 104
103.5 104
103.5 104
103.5 104
103.5 104
103.75 104
cut
104.0 105
104.0 105
104.0 105
104.0 105
104.0 105
104.25 105
104.5 105
104.5 105
104.5 105
104.5 105
104.5 105
104.75 105
cut
105.0 106
105.0 106
105.0 106
105.0 106
105.25 106
105.5 106
105.5 106
105.5 106
105.5 106
105.75 106
cut
106.0 107
106.0 107
106.0 107
106.0 107
106.0 107
106.25 107
106.5 107
106.5 107
106.5 107
106.5 107
106.5 107
106.75 107
cut
107.0 108
107.0 108
107.0 108
107.0 108
107.25 108
107.5 108
107.5 108
107.5 108
107.5 108
107.5 108
107.75 108
cut
108.0

IndexError: single positional indexer is out-of-bounds

In [55]:
for k in raw_song3:
    print(k)

['0', ' 0', ' Header', ' 1', ' 5', ' 480']
['1', ' 0', ' Start_track']
['1', ' 0', ' Time_signature', ' 3', ' 2', ' 24', ' 8']
['1', ' 0', ' Key_signature', ' -1', ' "major"']
['1', ' 0', ' Tempo', ' 769231']
['1', ' 0', ' Control_c', ' 0', ' 121', ' 0']
['1', ' 0', ' Program_c', ' 0', ' 52']
['1', ' 0', ' Control_c', ' 0', ' 7', ' 100']
['1', ' 0', ' Control_c', ' 0', ' 10', ' 64']
['1', ' 0', ' Control_c', ' 0', ' 91', ' 30']
['1', ' 0', ' Control_c', ' 0', ' 93', ' 30']
['1', ' 0', ' MIDI_port', ' 0']
['1', ' 0', ' Note_on_c', ' 0', ' 60', ' 80']
['1', ' 0', ' Note_on_c', ' 0', ' 69', ' 80']
['1', ' 0', ' Note_on_c', ' 0', ' 60', ' 80']
['1', ' 719', ' Note_on_c', ' 0', ' 60', ' 0']
['1', ' 719', ' Note_on_c', ' 0', ' 69', ' 0']
['1', ' 719', ' Note_on_c', ' 0', ' 60', ' 0']
['1', ' 720', ' Note_on_c', ' 0', ' 62', ' 80']
['1', ' 720', ' Note_on_c', ' 0', ' 70', ' 80']
['1', ' 720', ' Note_on_c', ' 0', ' 62', ' 80']
['1', ' 959', ' Note_on_c', ' 0', ' 62', ' 0']
['1', ' 959', ' Note

In [226]:
for filename in os.listdir('chopin'):
    out=""
    song = py_midicsv.midi_to_csv("chopin/" + filename)
    song = [e.strip().split(", ") for e in song]
    df = pd.DataFrame(song)
    out += filename+" "
    out += df[ df[2] == "Key_signature"][3].values[0]
    print(out)

chpn-p1.mid 0
chpn-p10.mid 4
chpn-p11.mid 5
chpn-p12.mid 5
chpn-p13.mid 6
chpn-p14.mid -6
chpn-p15.mid -5
chpn-p16.mid -5
chpn-p17.mid -4
chpn-p18.mid -4
chpn-p19.mid -3
chpn-p2.mid 0
chpn-p20.mid -3
chpn-p21.mid -2
chpn-p22.mid -2
chpn-p23.mid -1
chpn-p24.mid -1
chpn-p3.mid 1
chpn-p4.mid 1
chpn-p5.mid 2
chpn-p6.mid 2
chpn-p7.mid 3
chpn-p8.mid 3
chpn-p9.mid 4
chpn_op10_e01.mid 0
chpn_op10_e05.mid -6
chpn_op10_e12.mid -3
chpn_op23.mid -2
chpn_op25_e1.mid -4
chpn_op25_e11.mid 0
chpn_op25_e12.mid -3
chpn_op25_e2.mid -4
chpn_op25_e3.mid -1
chpn_op25_e4.mid 0
chpn_op27_1.mid 4
chpn_op27_2.mid -5
chpn_op33_2.mid 2
chpn_op33_4.mid 2
chpn_op35_1.mid -5
chpn_op35_2.mid -6
chpn_op35_3.mid -5
chpn_op35_4.mid -5
chpn_op53.mid -4
chpn_op66.mid 4
chpn_op7_1.mid -2
chpn_op7_2.mid 0
chp_op18.mid -3
chp_op31.mid -5
Silent_Night_-_Pentatonic_ver..mid -1


In [18]:
def showA(A,a,b):
#     for i in range(len(arr ) ):
#         out = ""
#         for j in range (97):
#             out += str(int(arr[i][j])) + " "
#         print(out)

    X,Y = np.meshgrid(np.arange(A.shape[1]), np.arange(A.shape[0]))
    plt.scatter(X.flatten(), Y.flatten(), c=A.flatten())
    plt.savefig('plot/'+str(a)+"_"+str(b)+".png")
    plt.show()

In [352]:
a = 1.9954
(5.01).is_integer()

False